In [2]:
import torch
import sys
sys.path.append('/home/cyang314/ucr_work/add-thin')

from add_thin.metrics import forecast_wasserstein
from add_thin.evaluate_utils import get_task, get_run_data

In [3]:
# Set run id and paths
RUN_ID = "naflz4ti"

WANDB_DIR = "/home/cyang314/ucr_work/add-thin/wandb"
PROJECT_ROOT = "/home/cyang314/ucr_work/add-thin/"  # should include data folder

In [4]:
def sample_model(task, datamodule):
    """
    Sample forecasts from model.
    """
    samples = []
    targets = []
    mapes = []
    with torch.no_grad():
        for batch in datamodule.test_dataloader():
            batch.to(task.device)
            # Sample 50 forecasts
            for _ in range(10):
                # Set history
                future, tmax, tmin = task.set_history(
                    batch.concat(batch, batch, batch, batch)
                )  # Note that we are using the same batch 5 times to get 5 different histories

                # Sample forecasts from model
                sample = task.model.sample(
                    len(future),
                    tmax=future.tmax,
                )

                # Rescale and shift to right forecast window
                sample.time = (sample.time / future.tmax) * (tmax - tmin)[
                    :, None
                ] + tmin[:, None]

                # Calculate Absolute Percentage Error
                mapes.append(
                    (
                        torch.abs(future.mask.sum(-1) - sample.mask.sum(-1))
                        / (future.mask.sum(-1) + 1)
                    )
                    .detach()
                    .cpu()
                )

                samples = samples + sample.to_time_list()
                targets = targets + future.to_time_list()

    return samples, targets, mapes

In [5]:
# Get run data
data_name, seed, run_path = get_run_data(RUN_ID, WANDB_DIR)

# Get task and datamodule
task, datamodule = get_task(run_path, density=False, data_root=PROJECT_ROOT)

# Sample forecasts
samples, targets, mapes = sample_model(task, datamodule)

# Calculate Wasserstein distance and MAPE
wassertstein_distance = forecast_wasserstein(
    samples,
    targets,
    datamodule.tmax.detach().cpu().item(),
)
MAPE = torch.cat(mapes).mean()

# Print rounded results for data and seed
print("ADD and Thin forecast evaluation:")
print("================================")
print(
    f"{data_name} (Seed: {seed}): Wasserstein: {wassertstein_distance:.3f}, MAPE: {MAPE:.3f}"
)

/home/cyang314/ucr_work/add-thin/add_thin/data.py:659: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loader = torch.load(path, map_location=device)
/home/cyang314/.conda/env

AttributeError: 'AddThin' object has no attribute 'forecast_window'